In [45]:
from numpy import zeros, array
from functools import lru_cache
from math import log
from tqdm import tqdm

from random import randint as rint
from random import seed, shuffle
seed(42)

import pandas as pd
import pymorphy2
import time
import re

In [147]:
class Node(object):
    def __init__(self, value, child=None, key=None, end=False):
        if child is not None and key is not None:
            self.children = {key: child}
        else:
            self.children = {}
        self.value = value
        self._isEnd = end

    def __str__(self):
        return f"keys = {list(self.children.keys())}, value = {self.value}, \
end = {self._isEnd}"

    def addChild(self, value, key):
        self.children[key] = Node(value)

    def hasChild(self, key):
        if key in self.children.keys():
            return True
        return False

    def getChild(self, key):
        return self.children[key]

    def setValue(self, value):
        self.value = value

    def getValue(self):
        return self.value

    def isEnd(self):
        return self._isEnd

    def getKeys(self):
        return list(self.children.keys())

    def setEnd(self):
        self._isEnd = True


class PrefixTrie(object):
    def __init__(self, db=None):
        self.root = Node('')

        if db is not None:
            for word in db:
                self.insert(word, word)

    def insertDict(self, db_form, db_lemma):
        for i in range(len(db_form)):
            self.insert(db_form[i], db_lemma[i])

    def insert(self, key, value):
        node = self.root
        lenght_key = len(key)
        way = ""

        for i in range(lenght_key):
            char = key[i]
            way += char
            if not node.hasChild(char):
                node.addChild(way, char)
            node = node.getChild(char)
        if (not node.isEnd()):
            node.setValue(value)
            node.setEnd()

    def hasKey(self, key):
        node = self.root

        for i in range(len(key)):
            char = key[i]
            if node.hasChild(char):
                node = node.getChild(char)
            else:
                return False

        if node.isEnd():
            return True
        return False

    def largestPrefix(self, key):
        node = self.root
        lemma = ""
        lenght_key = len(key)

        for i in range(lenght_key):
            char = key[i]
            if not node.hasChild(char):
                break
            node = node.getChild(char)
            if node.isEnd():
                lemma = node.getValue()

        dif = lenght_key - len(lemma)
        if dif != 0:
            depth = 1
            stack = [node.getChild(i) for i in node.getKeys()]
            next_stack = []

            while depth <= max(dif, int(lenght_key / 2)):
                for elem in stack:
                    if elem.isEnd():
                        return elem.getValue()
                    next_stack += [elem.getChild(i) for i in elem.getKeys()]

                stack = next_stack.copy()
                next_stack = []
                depth += 1

        return lemma if lemma != "" else key

    @lru_cache
    def getLemma(self, word):
        prefix = ""
        lemma = word
        len_word = len(word)
        min_dist = len_word

        for i in range(len_word):
            prefix += word[i]
            new_lemma = self.largestPrefix(prefix)
            dist = lev_dist(new_lemma, word)

            if dist < min_dist and dist < len_word:
                if dist == 0:
                    return new_lemma
                lemma = new_lemma
                min_dist = dist

        return lemma

In [2]:
lemma_dict = pd.read_csv('C:\\Python programs\\ru_dict_lemma.csv')
print(lemma_dict)

         lemma
0            а
1           аа
2          а-а
3          ааа
4        а-а-а
...        ...
51728  ящерица
51729   ящерка
51730     ящик
51731   ящичек
51732     ящур

[51733 rows x 1 columns]


In [5]:
df_train = pd.read_csv('C:\\Python programs\\dataset_ru_word_lemma2.csv')
print(df_train)

               form          lemma  id_sen
0                 «              «     0.0
1              Если           если     0.0
2          передача       передача     0.0
3          цифровых       цифровой     0.0
4        технологий     технология     0.0
...             ...            ...     ...
19350  провозглашал  провозглашать   999.0
19351          себя           себя   999.0
19352        другом           друг   999.0
19353          мира            мир   999.0
19354             .              .   999.0

[19355 rows x 3 columns]


In [28]:
df_stopwords = pd.read_csv('C:\\Python programs\\stopwords_ru.csv')
print(df_stopwords)

            stopword
0                  а
1          абсолютно
2     авторизоваться
3           активный
4               алло
...              ...
1367               „
1368               “
1369               …
1370               /
1371             ...

[1372 rows x 1 columns]


In [23]:
@lru_cache
def lev_dist(w1, w2):
    n, m = len(w1), len(w2)
    mat = zeros([n + 1, m + 1], int)
    mat[0, :] = array([i for i in range(m + 1)])
    mat[:, 0] = array([i for i in range(n + 1)])

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            mat[i][j] = min(mat[i - 1][j] + 1, mat[i][j - 1] + 1,
                            mat[i - 1][j - 1] + (w1[i - 1] != w2[j - 1]))
    return mat[n][m]

In [101]:
def lemma_lev(w):
    lemma = ""
    len_w = len(w)
    len_lemma = len_w
    for word in lemma_dict["lemma"]:
        if abs(len_w - len(word)) >= len_lemma:
            continue
            
        word = word.lower()
        dist = lev_dist(word, w)
        if dist < len_lemma:
            if dist == 0:
                return word
            lemma = word
            len_lemma = dist

    return lemma if lemma != "" else w

In [134]:
def test_metric(metric, cnt_test=100):
    cnt_test = min(cnt_test, len(df_train.index))
    acc = 0
    num_reg_exp = r'[-+]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][-+]?\d+)?'
    
    trie_st = PrefixTrie(df_stopwords['stopword'])
    trie = PrefixTrie(lemma_dict["lemma"].apply(str.lower))
    
    test_word, test_lemms = [], []
    rand_idx = list(df_train.index)
    shuffle(rand_idx)
    
    for i in rand_idx:
        word = df_train['form'][i].lower()
        lemma = df_train['lemma'][i].lower()
        if trie_st.hasKey(word) or trie_st.hasKey(lemma) or re.fullmatch(num_reg_exp, word) is not None or not trie.hasKey(lemma):
            continue
        
        test_word += [word]
        test_lemms += [lemma]
        
        if len(test_word) == cnt_test:
            break
            
    cnt_test = len(test_word)
    
    with tqdm(total=cnt_test, position=0, leave=True) as pbar:
        for word, ans_lemma in zip(test_word, test_lemms):
            
            ans = metric(word)
            if ans == ans_lemma:
                acc += 1
            
            pbar.set_description(f"Word: {word}, Lemma: {ans_lemma}, Ans: {ans}")
            pbar.update()
            
    print(acc / cnt_test)

In [110]:
def jaro_dist(w1, w2):
    len_w1, len_w2 = len(w1), len(w2)
    blok = max(len_w1, len_w2) // 2 - 1
    tr = 0
    m = 0
    
    for i in range(len_w1):
        char1 = w1[i]
        for j in range(len_w2):
            if abs(i - j) <= blok:
                if w2[j] == char1:
                    m += 1
                    if i != j:
                        tr += 1
                    break
    tr //= 2             
    return 0 if m == 0 else 1/3 * (m / len_w1 + m / len_w2 + (m - tr) / m)          

In [152]:
def lemma_jaro(w):
    lemma = w
    len_w = len(w)
    max_dist = 0
    for word in lemma_dict["lemma"]:
        word = word.lower()
        if abs(len_w - len(word)) >= len_w:
            continue

        dist = jaro_dist(word, w)
        if dist > max_dist and dist <= 1:
            if dist == 1:
                return word
            lemma = word
            max_dist = dist

    return lemma

In [153]:
lemma_jaro('гнездился')

'гнездиться'

In [148]:
trie = PrefixTrie(lemma_dict["lemma"].apply(str.lower))

In [136]:
@lru_cache
def lemma_dbsra(w):
    lemma = ""
    for i in range(len(w)):
        for j in range(len(w), i, -1):
            if trie.hasKey(w[i:j]) and j - i > len(lemma):
                lemma = w[i:j]
    return lemma if lemma != "" else w

In [139]:
def damerau_levenshtein_distance(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i] == s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min(d[(i,j)], d[i-2,j-2] + 1) # transposition
 
    return d[lenstr1-1,lenstr2-1]

In [140]:
def lemma_lev_dam(w):
    lemma = w
    len_w = len(w)
    min_dist = len_w
    for word in lemma_dict["lemma"]:
        word = word.lower()
        if abs(len_w - len(word)) >= len_w:
            continue

        dist = damerau_levenshtein_distance(word, w)
        if dist < min_dist:
            if dist == 0:
                return word
            lemma = word
            min_dist = dist

    return lemma

In [151]:
test_metric(lemma_jaro, 200)
# trie.getLemma: 0.800
# lemma_lev: 0.775
# lemma_lev_dam: 0.740
# lemma_jaro: 0.655
# lemma_dbsra: 0.392
# lambda x: x: 0.240

Word: сша, Lemma: сша, Ans: сша:  20%|█████████▎                                      | 39/200 [00:34<01:51,  1.45it/s]

саша сша


Word: целях, Lemma: цель, Ans: лелеять:  32%|█████████████                            | 64/200 [00:52<01:25,  1.59it/s]

приемлемый приемы


Word: сша, Lemma: сша, Ans: сша:  33%|███████████████▊                                | 66/200 [00:55<02:05,  1.07it/s]

...

советовать совета


Word: потерять, Lemma: потерять, Ans: потереть:  92%|█████████████████████████████▌  | 185/200 [02:44<00:12,  1.16it/s]

перепроверять потерять


Word: поводу, Lemma: повод, Ans: повод: 100%|████████████████████████████████████████| 200/200 [02:56<00:00,  1.13it/s]

0.645
